In [141]:
import pandas as pd
import datetime

In [142]:
class SOT(object):
    def __init__(self, sot_file):
        self.sot_file = pd.DataFrame.from_csv(sot_file)
        
    def save_csv(self):
        print(datetime.datetime.now().strftime("%Y-%m-%d-%H:%M"))
    
    def update_budget(self, budget_records):
        return budget_records

    def update_bank(self, bank_records):
        pass

sot = SOT('first_SOT.csv')


In [163]:
class Test_sot(object):
    def __init__(self):
        starter_budget = {'uA':['a','d','e','z','multiplematch'],
              'uB':[3,4,5,6,9],
              'uk1':[33,44,66,99,99]}

        starter_bank = {'aA':['b','d','e', 'l'],
                      'aB':[3,4,6,9],
                      'ak1':[33,44,77,88]}

        budget_df = pd.DataFrame(data=starter_budget)
        bank_df = pd.DataFrame(data=starter_bank)

        self.starter_df = pd.merge(bank_df, budget_df, how='outer', left_on='ak1',right_on='uk1')
    
    def bank_row_type(self, key, bank_df, updated_sot):
        match_bank_cols = updated_sot['ak1'] == key
        matched_bank = match_bank_cols.sum()
        assert matched_bank < 2, 'multiple match for bank - check function to match bank with bank'
        if not matched_bank:
            # New bank entry...
            matched_budget = (updated_sot['uk1'] == key)
            number_budget_matches = matched_budget.sum()
            if number_budget_matches:
                # Matches 
                if number_budget_matches == 1:
                    # found unique match
                    match = updated_sot[matched_budget].index
                    assert len(match) == 1, 'match is not unique'
                    return 'unique match', match[0]
                elif number_budget_matches > 1:
                    # found multiple matches
                    return 'multiple match'
            else:
                # Need to make new row
                return 'brand new'
        else:
            # Entry already found in bank section, no need to do anything
            return 'existing bank match'
        
    def bank_row_type_strategy(self, row_type, row, second_bank_df, updated_sot):
        a,b,key = row
        bank_cols = second_bank_df.columns
        if row_type == 'existing bank match':
            return updated_sot  # Do nothing
        elif row_type == 'brand new':
            new_row = pd.DataFrame(data=row, index=bank_cols).transpose()
            return pd.concat([updated_sot, new_row])  # Add new row to updated_df
        elif row_type[0] == 'unique match':
            correct_row = updated_sot
            for i in range(len(row)):
                updated_sot = updated_sot.set_value(row_type[1], bank_cols[i],row[i])
            return updated_sot  # Match with 
        elif row_type == 'multiple match':
            print('multiple match: {}'.format(row))
            return updated_sot  # Do nothing
        else:
            raise Exception('unhandled case of bank row type')
                
    
    def update_bank(self):
        updated_sot = self.starter_df
        second_bank = {'aA':['b', 'g', 'd','e','z','lolol'],
              'aB':[3,4,5,6,99,70],
              'ak1':[33,66,77,111,888,99]}
        second_bank_df = pd.DataFrame(data=second_bank)
        for n, row in second_bank_df.iterrows():
            a, b, key = row
            row_type = self.bank_row_type(key, second_bank_df, updated_sot)
            updated_sot = self.bank_row_type_strategy(row_type, row, second_bank_df, updated_sot)
        return updated_sot

        
t = Test_sot()

In [164]:
t.update_bank()

multiple match: aA     lolol
aB        70
ak1       99
Name: 5, dtype: object


,aA,aB,ak1,uA,uB,uk1
0,b,3,33,a,3,33
1,d,4,44,d,4,44
2,e,6,77,NaN,NaN,NaN
3,l,9,88,NaN,NaN,NaN
4,g,4,66,e,5,66
5,NaN,NaN,NaN,z,6,99
6,NaN,NaN,NaN,multiplematch,9,99
3,e,6,111,NaN,NaN,NaN
4,z,99,888,NaN,NaN,NaN
